In [1]:
# Wavefunction generation
from pyscf import mcscf, fci, lib 
from pyscf import gto, scf, tools, dft
import api as pyq 
import importlib
import os
import h5py
import pandas as pd
import pdb
import pyscf
import numpy as np
import pyqmc
import scipy
import copy
import gpu
import matplotlib.pyplot as plt
from pyscf.scf.hf import dump_scf_summary
from concurrent.futures import ProcessPoolExecutor
from pyscf.scf.chkfile import dump_scf
print(pyq.__file__)
print(pyscf.__file__)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/__init__.py


In [2]:
# Wavefunction generation
from scf_runs import run_lda_he, run_casci
scf_checkfile, opt_checkfile, mf_lda = run_lda_he()
ci_checkfile, mc, opt_chkfile, vmc_chkfile = run_casci(scf_checkfile, nroots=1, ncas = 2)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/__init__.py
He atom neutral LDA spin=0
HF
converged SCF energy = -2.86151422722833  <S^2> = 2.4424907e-15  2S+1 = 1
LDA
converged SCF energy = -2.83456474290722  <S^2> = 3.1086245e-15  2S+1 = 1
CASCI nelecas up/down (1, 1)
CASCI E = -2.86531899726653  E(CI) = -2.86531899726653  S^2 = 0.0000000
Available output from CASCI: <KeysViewHDF5 ['ci', 'fci', 'mc_mo_energy', 'mf_mo_energy', 'mo_coeff', 'mo_occ', 'ncas', 'ncore', 'nelecas']>


In [3]:
import importlib, recipes, bosonrecipes, bosonslater, bosonaccumulators, bosonwftools, bosonslater_wf_based
modules = [recipes, bosonrecipes, bosonrecipes, bosonslater, bosonaccumulators, bosonwftools] # Reloads recipes.py file after each update
for m in modules:
    importlib.reload(m)


wf = bosonslater.BosonWF(mf_lda.mol, mf_lda, mc)
wf_wfb = bosonslater_wf_based.BosonWF(mf_lda.mol, mf_lda, mc)
configs = bosonrecipes.initial_guess(mf_lda.mol, 100)

In [4]:
configs.configs.shape

(100, 2, 3)

In [5]:
tol = 1E-1

nconf, nelec, _ = configs.configs.shape

def is_equal(a, b, tag=None):
    result = (np.abs(a-b) < tol).all()
    if not result:
        print(tag, result, 'diff', np.max(np.abs(a-b)), 'a', np.max(np.abs(a)), 'b', np.max(np.abs(b)))
    else:
        print(tag, result)
    return result
    
def limdrift(g, cutoff=1):
    """
    Limit a vector to have a maximum magnitude of cutoff while maintaining direction

    :parameter g: a [nconf,ndim] vector
    :parameter cutoff: the maximum magnitude
    :returns: The vector with the cutoff applied.
    """
    tot = np.linalg.norm(g, axis=1)
    mask = tot > cutoff
    g[mask, :] = cutoff * g[mask, :] / tot[mask, np.newaxis]
    return g


In [16]:
nsteps = 10
nelec = 2
tstep = 0.3
for _ in range(nsteps):
    acc = 0.0
    for e in range(nelec):
        import pdb
        sign_p,  val_p   = wf.recompute(configs)
        sign_p2, val_p2  = wf_wfb.recompute(configs)
        is_equal(val_p, val_p2, 'val')
        
        wf_k = copy.deepcopy(wf)
        wf_wfb_k = copy.deepcopy(wf_wfb)

        g, _, _ = wf.gradient_value(e, configs.electron(e))
        g_wfb, _, _ = wf_wfb.gradient_value(e, configs.electron(e))
        is_equal(g, g_wfb, 'g')
        
        grad = limdrift(np.real(g.T)) 
        grad_wfb = limdrift(np.real(g_wfb.T)) 
        is_equal(g, g_wfb, 'g')
        is_equal(grad, grad_wfb, 'grad')
        
        gauss = np.random.normal(scale=np.sqrt(tstep), size=(nconf, 3))
        newcoorde = configs.configs[:, e, :] + gauss + grad * tstep
        newcoorde = configs.make_irreducible(e, newcoorde)

        # # Compute reverse move
        g_k, _, saved = wf_k.gradient_value(e, newcoorde)
        g_k_wfb, _, _ = wf_wfb_k.gradient_value(e, newcoorde)

        is_equal(g_k, g_k_wfb, 'g_k')
        new_grad = limdrift(np.real(g_k.T))
        new_grad_wfb = limdrift(np.real(g_k_wfb.T))

        forward = np.sum(gauss**2, axis=1)
        backward = np.sum((gauss + tstep * (grad + new_grad)) ** 2, axis=1)
        backward_wfb = np.sum((gauss + tstep * (grad_wfb + new_grad_wfb)) ** 2, axis=1)

        is_equal(backward, backward_wfb, 'backward')
        
        t_prob = np.exp(1 / (2 * tstep) * (forward - backward))
        t_prob_wfb = np.exp(1 / (2 * tstep) * (forward - backward_wfb))
        is_equal(t_prob, t_prob_wfb, 't_prob')

        # ratio = np.exp(2*(val_new-val_old))* t_prob
        newcoord = copy.deepcopy(configs)
        newcoord.configs[:,e,:] = newcoorde.configs
        
        _, val_k = wf_k.recompute(newcoord)
        _, val_k_wfb = wf_k.recompute(newcoord)

        ratio = np.exp(2*(val_k-val_k_wfb))* t_prob
        
        accept = ratio > np.random.rand(nconf)
        
        configs.move(e, newcoorde, accept)
        wf.updateinternals(e, newcoorde, configs, mask=accept, saved_values=saved)
        wf_wfb.updateinternals(e, newcoorde, configs, mask=accept, saved_values=saved)
        


val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backward True
t_prob True
val True
g True
g True
grad True
g_k True
backwa